<img src="https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/media/logo/newebac_logo_black_half.png" alt="ebac-logo">

---

# **Módulo** | Análise de Dados: Análise Exploratória de Dados de Logística II
Caderno de **Exercícios**<br>
Professor [André Perez](https://www.linkedin.com/in/andremarcosperez/)

---

# **Tópicos**

<ol type="1">
  <li>Manipulação;</li>
  <li>Visualização;</li>
  <li>Storytelling.</li>
</ol>


---

# **Exercícios**

Este *notebook* deve servir como um guia para **você continuar** a construção da sua própria análise exploratória de dados. Fique a vontate para copiar os códigos da aula mas busque explorar os dados ao máximo. Por fim, publique seu *notebook* no [Kaggle](https://www.kaggle.com/).

---

# **Análise Exploratória de Dados de Logística**

## 1\. Contexto

Neste projeto iremos analisar dados de logistica da empresa Loggi. A empresa trabalha com entregas de produtos diversos em todo Brasil.

Aqui iremos gerar insights sobre os problemas enfrentados na cidade de Brasil-DF. Problemas como: otimização das rotas de entrega, alocação de entregas nos veículos da frota com capacidade limitada, etc.

## 2\. Pacotes e bibliotecas

In [1]:
!pip install geopandas

In [2]:
# Todas as bibliotecas usadas

import json

import pandas as pd
import geopandas
import seaborn as sns
import numpy as np
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt

In [3]:
# download dos dados usados

# Coleta de dados das entregas;
!wget -q "https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/deliveries.json" -O deliveries.json
# Coleta de dados de geolocalização reversa das entregas. Feita localmente devido a quantidade exorbitante de conteúdo para as regras do Nominatin online
!wget -q "https://raw.githubusercontent.com/andre-marcos-perez/ebac-course-utils/main/dataset/deliveries-geodata.csv" -O deliveries-geodata.csv
# Coleta dos dados do mapa do Distrito Federal
!wget -q "https://geoftp.ibge.gov.br/cartas_e_mapas/bases_cartograficas_continuas/bc100/go_df/versao2016/shapefile/bc100_go_df_shp.zip" -O distrito-federal.zip
!unzip -q distrito-federal.zip -d ./maps
!cp ./maps/LIM_Unidade_Federacao_A.shp ./distrito-federal.shp
!cp ./maps/LIM_Unidade_Federacao_A.shx ./distrito-federal.shx

[distrito-federal.zip]
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of distrito-federal.zip or
        distrito-federal.zip.zip, and cannot find distrito-federal.zip.ZIP, period.
cp: cannot stat './maps/LIM_Unidade_Federacao_A.shp': No such file or directory
cp: cannot stat './maps/LIM_Unidade_Federacao_A.shx': No such file or directory


## 3\. Exploração de dados

In [4]:
# carregando os dados do arquivo em um dicionário Python chamado data
with open('deliveries.json', mode='r', encoding='utf8') as file:
  data = json.load(file)

# - wrangling da estrutura;
deliveries_df = pd.DataFrame(data) # Criando um dataframe pandas com os dados json coletos da Loggi

# Normalizando os dados da coluna 'origin' que estão em forma de dicionário
hub_origin_df = pd.json_normalize(deliveries_df['origin']) # carregando os dados normalizados em um dataframe chamamdo 'hub_origin_df'.
deliveries_df = pd.merge(deliveries_df, hub_origin_df, how='right', left_index=True, right_index=True) # Combinando 'hub_origin_df' com o dataframe original
deliveries_df = deliveries_df.drop('origin', axis=1) # Removendo a coluna que foi normalizada em outras duas colunas.
deliveries_df = deliveries_df[['name', 'region', 'lng', 'lat', 'vehicle_capacity', 'deliveries']] # organizando o dataframe para uma melhor leitura.
deliveries_df.rename(columns={'lng': 'hub_lng', 'lat': 'hub_lat'}, inplace=True) # renomeando as novas colunas para melhor leitura.

# A coluna 'deliveries' está em formato de lista de dicionários de dicionários. Portanto vamos dar um explode para melhor visualisação
deliveries_exploded_df = deliveries_df[['deliveries']].explode('deliveries') # carregando o dataframe explodido na variavel 'deliveries_exploded_df'
deliveries_normalized_df = pd.concat([
    pd.DataFrame(deliveries_exploded_df['deliveries'].apply(lambda x: x['size'])).rename(columns={'deliveries': 'dlvr_size'}),
    pd.DataFrame(deliveries_exploded_df['deliveries'].apply(lambda x: x['point']['lng'])).rename(columns={'deliveries': 'dlvr_lng'}),
    pd.DataFrame(deliveries_exploded_df['deliveries'].apply(lambda x: x['point']['lat'])).rename(columns={'deliveries': 'dlvr_lat'})
], axis=1) # Normalizando os dados da coluna 'deliveries' do nosso dataframe explodido que estão em forma de dicionários
deliveries_df = deliveries_df.drop('deliveries', axis=1) # removendo a coluna original 'deliveries' que já foi explodida e normalizada
deliveries_df = pd.merge(deliveries_df, deliveries_normalized_df, how='right', left_index=True, right_index=True) # Combinando o dataframe normalizado com o dataframe original prezervando o index de cada instancia
deliveries_df.reset_index(inplace=True, drop=True) # Resetando o indice de cada linha para obtermos o tamanho real do dataframe, ou seja, o número total de entregas.

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

Exploração do schema;

In [ ]:
deliveries_df # DataFrame final com todos os dados normalizados

In [ ]:
deliveries_df.dtypes # Colunas e seus respectivos tipos de dados

In [ ]:
deliveries_df.select_dtypes('object').describe().transpose() # Atribudos categóricos

In [ ]:
deliveries_df.select_dtypes('int64').describe().transpose() # Atribudos numéricos

In [ ]:
deliveries_df.isna().any() # Verificando quais colunas possuem dados faltantes

## 4\. Manipulação

In [ ]:
# - enriquecimento;
# Geocodificação dos hubs de distribuição
hub_df = deliveries_df[['region', 'hub_lng', 'hub_lat']] # Carregando um dataframe chamado 'hub_df' com as colunas de interesse.
hub_df = hub_df.drop_duplicates().sort_values(by='region').reset_index(drop=True) # removendo os valores duplicados para obtermos as coordenadas de cada um dos 3 hubs existentes

# Configuração do Nominatim seguindo as regras de uma consulta por segundo
geolocator = Nominatim(user_agent='ebac_geocoder')
geocoder = RateLimiter(geolocator.reverse, min_delay_seconds=1)

# Adicionando colunas ao dataframe 'hub_df' com o conteudo de suas coordenadas e geolocalização reversa.
hub_df['coordinates'] = hub_df['hub_lat'].astype(str) + ', ' + hub_df['hub_lng'].astype(str) # coluna com as coordenadas geograficas em forma de string
hub_df['geodata'] = hub_df['coordinates'].apply(geocoder) # coluna com o conteúdo da geolocalização reversa das coordenadas.

# Normalizando a coluna 'geodata' extraindo um novo dataframe com o endereço de cada hub devidademente dividido
hub_geodata_df = pd.json_normalize(hub_df['geodata'].apply(lambda x: x.raw))
hub_geodata_df.rename(columns={'address.town': 'hub_town', 'address.suburb': 'hub_suburb', 'address.city': 'hub_city'}, inplace=True) # renomeando colunas de interesse para melhor leitura
hub_geodata_df['hub_city'] = np.where(hub_geodata_df['hub_city'].notna(), hub_geodata_df['hub_city'], hub_geodata_df['hub_town']) # Trazendo para a coluna 'hub_city' o que tem de duplicado com a coluna 'hub_town', que para esse caso, pode significar a mesma coisa.
hub_geodata_df['hub_suburb'] = np.where(hub_geodata_df['hub_suburb'].notna(), hub_geodata_df['hub_suburb'], hub_geodata_df['hub_city']) # Trazendo apra a coluna 'hub_suburb' o que tem de duplicado com a coluna 'hub_city', que para esse caso, pode significar a mesma coisa.
hub_geodata_df = hub_geodata_df.drop('hub_town', axis=1) # removendo a coluna 'hub_town' para melhor leitura do endereço

# Combinando o dataframe de endereço do hub com o dataframe original das entregas
hub_df = pd.merge(hub_df, hub_geodata_df, left_index=True, right_index=True) # Combinando o dataframe de endereço do hub com o dataframe dos hubs de distribuição
hub_df = hub_df[['region', 'hub_suburb', 'hub_city']] # organizando o dataframe dos hubs de distribuição formatado para melhor leitura
deliveries_df = pd.merge(deliveries_df, hub_df, how='inner', on='region') # Combinando o dataframe de hubs formatado com o dataframe original
deliveries_df = deliveries_df[['name', 'region', 'hub_lng', 'hub_lat', 'hub_city', 'hub_suburb', 'vehicle_capacity', 'dlvr_size', 'dlvr_lng', 'dlvr_lat']] # Organizando o dataframe para melhor leitura

# - controle de qualidade;
# - etc.

In [ ]:
# Carregando o arquivo de geolocalização em um dataframe pandas
deliveries_geodata_df = pd.read_csv('deliveries-geodata.csv')
deliveries_df = pd.merge(deliveries_df,
                         deliveries_geodata_df[['delivery_city', 'delivery_suburb']],
                         how='inner', left_index=True, right_index=True) # Comninando o dataframe de geodata deliveries com o dataframe orginal


## **Analisando a qualidade dos dados extraídos até agora.**

In [ ]:
deliveries_df.info()

## **Analisando quais colunas existem valores faltantes**

In [ ]:
deliveries_df.isna().any()

## **Percentagem de valores faltantes nas cidades de cada entrega**

In [ ]:
city_na = 100 * (deliveries_df["delivery_city"].isna().sum() / len(deliveries_df))
city_na

## **Percentagem de valores faltantes nos bairros das entregas**

In [ ]:
suburb_na = 100 * (deliveries_df["delivery_suburb"].isna().sum() / len(deliveries_df))
suburb_na

## **Percentagem de entregas em cada cidade**

In [ ]:
prop_city = 100 * (deliveries_df[['delivery_city']].value_counts() / len(deliveries_df))
prop_city = prop_city.sort_values(ascending=False)
prop_city

## **Percentagem de entregas em cada bairro**

In [ ]:
prop_df = deliveries_df[["delivery_suburb"]].value_counts() / len(deliveries_df)
prop_df.sort_values(ascending=False).head(30)

###    ***Percentagem de entregas por região***



In [ ]:
prop_region = 100 * (deliveries_df[['region']].value_counts() / len(deliveries_df))
prop_region

## 5\. Visualização

In [ ]:
# carregando o mapa do Distrito Federal na variavel mapa
mapa = geopandas.read_file('distrito-federal.shp')
mapa = mapa.loc[[0]]

In [ ]:
# Carregando a posição dos hubs de distribuição no dataframe 'geo_hub_df'
hub_df = deliveries_df[['region', 'hub_lng', 'hub_lat']].drop_duplicates().reset_index(drop=True) # Separando as coordenadas geograficas de cada hub
geo_hub_df = geopandas.GeoDataFrame(hub_df, geometry=geopandas.points_from_xy(hub_df['hub_lng'], hub_df['hub_lat'])) # adicionando a coluna 'geometry ao dataframe
geo_hub_df

In [ ]:
# Carregando a posição das entregas no dataframe 'geo_deliveries_df'
geo_deliveries_df = geopandas.GeoDataFrame(deliveries_df, geometry=geopandas.points_from_xy(deliveries_df['dlvr_lng'], deliveries_df['dlvr_lat']))
geo_deliveries_df

# Visualização do mapa geral de entregas por região

In [ ]:
# criando um plot vazio para melhor visualização
fig, ax = plt.subplots(figsize=(50/2.54, 50/2.54))

# mapa do distrito federal
mapa.plot(ax=ax, alpha=0.4, color='lightgrey')

# marcando cada entrega no mapa, por região
geo_deliveries_df.query('region ==  "df-0"').plot(ax=ax, markersize=0.5, color='red', label='df-0')
geo_deliveries_df.query('region == "df-1"').plot(ax=ax, markersize=0.5, color='blue', label='df-1')
geo_deliveries_df.query('region == "df-2"').plot(ax=ax, markersize=0.5, color='seagreen', label='df-2' )

# marcando cada hub no mapa
geo_hub_df.plot(ax=ax, markersize=30, marker='H', color='black', label='hub')

# configurando as legendas
plt.title('Entregas no Distrito Federal por região', fontdict={'fontsize': 20})
lgnd = plt.legend(prop={'size': 18})
for handle in lgnd.legendHandles:
  handle.set_sizes([50])

## Visualização de grafico de entrega por cidade.

In [ ]:
# Criando dataframe com a porcentagem de entrega por cidade
prop_city_df = prop_city.to_frame()
prop_city_df.rename(columns={0: 'city_percent'}, inplace=True)

# criando grafico de proporção de entregas por cidade
with sns.axes_style('whitegrid'):
  grafico = sns.barplot(data=prop_city_df.head(10), y='delivery_city', x='city_percent', ci=None, palette='dark')
  grafico.set(title='proporção de entregas por cidade', xlabel='Cidade', ylabel='Proporção');

# Visualização de grafico de entrega por região.

In [ ]:
# Criando dataframe com a porcentagem de entrega por região
data = pd.DataFrame(deliveries_df[['region', 'vehicle_capacity']].value_counts(normalize=True)).reset_index()
data.rename(columns={0: "region_percent"}, inplace=True)

# criando grafico de proporção de entregas por região
with sns.axes_style('whitegrid'):
  grafico = sns.barplot(data=data, x="region", y="region_percent", ci=None, palette="pastel")
  grafico.set(title='Proporção de entregas por região', xlabel='Região', ylabel='Proporção');

# **6. Insights Finais:**

De acordo com todas as informações coletadas, podemos perceber que existem 636149 entregas presentes, distribuidas em todo Distrito Federal através de 3 hubs de distribuição.
Para uma melhor logistica, podemos considerar a possibilidade da criação de mais hubs de distribuição posicionados estrategicamente. Por exemplo um hub a mais atendendo o que é df-1 e df-2, assim melhorando a rapidez do atendimento nas zonas mais populosas e com melhor poder aquisitivo da do distrito federal. E mais um hub menor, para atender toda a zona rural localizada na região df-0.
